In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 16 10:29:27 2017

@author: johank
"""

from __future__ import absolute_import, print_function
import numpy as np
import matplotlib.pyplot as plt
import pyopencl as cl
import os, time
from matplotlib.colors import LinearSegmentedColormap
import matplotlib
matplotlib.use('Qt4Agg')
path =  os.getcwd()

params = {'backend': 'Qt4Agg', # Qt4Agg ps
          'lines.markersize':15,
          'axes.labelsize': 22,
          'legend.fontsize': 22,
          'axes.titlesize' : 25,
          'font.size':22,
          'xtick.labelsize': 22,
          'ytick.labelsize': 22,
          'xtick.major.size': 5,
          'ytick.major.size': 5,
          'xtick.minor.size':2,
          'ytick.minor.size':2
          }
          
matplotlib.rcParams.update(params)
#matplotlib.rcParams['xtick.direction'] = 'in'
#matplotlib.rcParams['ytick.direction'] = 'in'

font = {'family': 'serif', #sans-serif monospace sans-serif
        'weight': 'normal',
        'size': 22,
        }
matplotlib.rc('font', **font)

# for Latex text if usetex=True
#plt.rc('text', usetex=True)

# %% Parameter definition 

# Defining the device that is used
DeviceNr = 1   # 0 = CPU; 1 = Intel; 2 = AMD 
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '0' # Set to 1 for compiler comments

# Algal Exchange parameters
Aup      =  1.1       # g/m3  Algal concentration in upper layer  Oosterschelde data
h        =  0.10      # m     Height of the lower layer  defined
f        =  100.0     # m3/m3/h  Phichange rate with upper layer  Guestimated

# Mussel update, growth & mortality parameters
c        =  0.1       # g/g/h Maximal consumption rate of the mussels  Riisgard 2001 
e        =  0.2       # g/g   Trophic efficiency of mussels  Lamie
dM       =  0.02      # g/g/h Density dependent mortality rate of the mussels  Calibrated
kM       =  150.0     # g/m2  Effect of density on mortality  Guestimated

# Spatial movement parameters
D        =  0.0005    # m2/h  The diffusion constant describing the movement of mussels
V        =  0.1*60*60 # m/h   Tidal advection constant(0.1 m/s * 60 sec * 60 min)

# The speeding constant Phi 
Phi      =  1000.0    # Speeding constant, accelerates mussel growth

# Simulation settings 
length   = 50.0       # Length of the physical landscape
Size     = 512        # Size of the 2D grid
BlockSize= 32         # Size of the workgroup with the GPU 

EndTime  = 180*24/Phi # Total simulation time
NumPlots = 100         # Number of times the figure is updated
dT       = 0.0002     # Time step
#计算了EndTime/dT=21600步？

# Precalculations
dx = length/Size      # Spatial step size
dy = length/Size      # Spatial step size

Grid_Width = Size
Grid_Height = Size

# %% Defining the initial values

A = np.zeros((Size*Size))+Aup
M = (np.random.rand(Size*Size) < 0.05) * 100.0+100.0

A_host = A.astype(np.float32)
M_host = M.astype(np.float32)

# Start the timer:
start_time = time.time()

# %% Reporting in the simulation on the console

print("");
print(" * * * * * * * * * * * * * * * * * * * * * * * * * * * * * ");
print(" * Mussel bed Patterns                                   * ");
print(" * PyOpenCL implementation : Johan van de Koppel, 2017   * ");
print(" * Following a model by Van de Koppel et al 2005         * ");
print(" * * * * * * * * * * * * * * * * * * * * * * * * * * * * * \n");

print(" Current grid dimensions: %d x %d cells\n" % (Grid_Width, Grid_Height));

# %% Setting up the OpenCL context

platform = cl.get_platforms()
Devices  = platform[0].get_devices()
context  = cl.Context([Devices[DeviceNr]])
queue    = cl.CommandQueue(context)

print(" Compute Device: %s\n" % Devices[DeviceNr].name)

mf = cl.mem_flags # Memory flags are set

# The array is allocated on the GPU and the initial values are copied onto it
A_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=A_host)
M_g = cl.Buffer(context, mf.READ_WRITE | mf.COPY_HOST_PTR, hostbuf=M_host)

# Set up simulation parameters
global_size=M_host.shape
local_size=(BlockSize,)

# %% Defining the OpenCL simulation kernel

# Parameters
Params = """
    #define Aup         %1.5f
    #define ff          %1.5f
    #define hh          %1.5f
    #define cc          %1.5f
    #define ee          %1.5f
    #define dM          %1.5f    
    #define kM          %1.5f
    #define V           %1.5f    
    #define D           %1.5f
    #define Phi         %1.5f
    #define dX          %1.5f
    #define dY          %1.5f    
    #define dT          %1.8f
    #define Grid_Width  %d
    #define Grid_Height %d\n\n
    """ % (Aup,f,h,c,e,dM,kM,V,D,Phi,dx,dy,dT,Grid_Width,Grid_Height)

ComputeCode = """

////////////////////////////////////////////////////////////////////////////////
// Laplacation operator definition, to calculate diffusive fluxes
////////////////////////////////////////////////////////////////////////////////

static float d2_dxy2(__global float* pop)
{
    // Getting thread coordinates on the grid
    size_t current = get_global_id(0);
    size_t row	 = (size_t)(current/Grid_Width);
    size_t column  = current%Grid_Width;
    
    // Computing positions of the neighbors
    size_t left    = row * Grid_Width + column-1;
    size_t right   = row * Grid_Width + column+1;
    size_t top     = (row-1) * Grid_Width + column;
    size_t bottom  = (row+1) * Grid_Width + column;
    
    float retval = ( (pop[left] - 2.0*pop[current] + pop[right]) )
                     /(float)dX/(float)dX +
                   ( (pop[top]  - 2.0*pop[current] + pop[bottom]))
                     /(float)dY/(float)dY;
    
    return retval;
}

///////////////////////////////////////////////////////////////////////////////
// Gradient operator definition, to calculate advective fluxes
///////////////////////////////////////////////////////////////////////////////

static float d_dy(__global float* pop)
{   
	// Getting thread coordinates on the grid
    size_t current = get_global_id(0);
    size_t row	 = (size_t)(current/Grid_Width);
    size_t column  = current%Grid_Width;
    
    size_t top=(row-1) * Grid_Width + column;        
    return ( pop[current] - pop[top] )/(float)dY ;
}

static float d_dx(__global float* pop)
{
	// Getting thread coordinates on the grid
    size_t current = get_global_id(0);
    size_t row	 = (size_t)(current/Grid_Width);
    size_t column  = current%Grid_Width;

	size_t left    = row * Grid_Width + column-1;	
	return (( pop[current] - pop[left] )/(float)dX );
}

///////////////////////////////////////////////////////////////////////////////
// Simulation kernel
///////////////////////////////////////////////////////////////////////////////

__kernel void SimulationKernel (__global float* A, __global float* M)
{
	
    size_t current  = get_global_id(0);
    size_t row      = floor((float)current/(float)Grid_Width);
    size_t column   = current%Grid_Width;
	
        	if (row > 0 && row < Grid_Width-1)
            {
        		float Consumption = cc * A[current] * M[current];
                
             float dAdt = (ff*(Aup-A[current]) - Consumption/hh - V*d_dy(A)); 
             float dMdt = (ee*Consumption - dM*M[current]*kM/(kM+M[current]) + D*d2_dxy2(M));
                
        		A[current]=A[current]+dAdt*dT;
        		M[current]=M[current]+Phi*dMdt*dT;
            }
            
        	// HANDLE Boundaries
        	else if(row==0)
        		//do copy of first row = second last row
            {
                A[current]=A[(Grid_Height-2)*Grid_Width+column];
                M[current]=M[(Grid_Height-2)*Grid_Width+column];
            }            
        	else if(row==Grid_Height-1)
        		//do copy of last row = second row
            {
                A[current]=A[1*Grid_Width+column];
                M[current]=M[1*Grid_Width+column];
            }            
         else if(column==0)
            {
                A[current]=A[row * Grid_Width + Grid_Width-2];
                M[current]=M[row * Grid_Width + Grid_Width-2];
            }
         else if(column==Grid_Width-1)
            {
                A[current]=A[row * Grid_Width + 1];
                M[current]=M[row * Grid_Width + 1];
            }
        
} // End SimulationKernel    
"""

# Here the kernel is compiled
program = cl.Program(context, Params + ComputeCode).build()

# %% Setting up the plot 

# Colormap definition for the mussels
MusselColors = [(0.70, 0.67, 0.62), (0.96, 0.91, 0.93), (0.64, 0.64, 0.71),\
                (0.44, 0.48, 0.56), (0.20, 0.27, 0.28), (0.0 , 0.0 , 0.0)]  
MusselMap = LinearSegmentedColormap.from_list('AridVeg', MusselColors, N=100)

# Colormap definition for the Algae
AlgaeColors = [(0.84, 1.0, 1.0), (0.25, 0.8, 0.5)] 
AlgaeMap = LinearSegmentedColormap.from_list('Water', AlgaeColors, N=100)

# Setting up the figure window, with two panels
plt.close("all")
f1, (ax1, ax2) = plt.subplots(1, 2)
f1.canvas.set_window_title("Klausmeier's vegetation model")

# Setting window size and location 
mngr = plt.get_current_fig_manager()
mngr.window.setGeometry(0,25,1440, 720)

text=f1.suptitle("Time: %1.0f of %1.0f" % (0.0, EndTime*Phi/24),
            y=0.1, fontsize=20);

# Panel 1: the algae
im1=ax1.imshow(M_host.reshape(Size,Size), cmap=MusselMap, clim=(0, 1200), extent=[0,length,0,length]);
ax1.set_title('Mussels M', fontsize=20)
cb1=f1.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
cb1.ax.set_title(r'g/m$^{2}$',fontsize=20,horizontalalignment='center')
ax1.set_xticks([]) 

ax1.set_ylabel(r'Scale (meters)')

# Panel 2: the mussels
im2=ax2.imshow(A_host.reshape(Size,Size), cmap=AlgaeMap, clim=(0, 1));
ax2.set_title('Algae A', fontsize=20)
cb2=f1.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
cb2.ax.set_title(r'g/m$^{3}$',fontsize=20,horizontalalignment='center')
ax2.set_xticks([]) 
ax2.set_yticks([]) 

# %% The Simulation loop

# Setting up time profiling
CompuTime = GraphTime = 0

# Starting the loop
for ii in range(NumPlots):
    
    TimePointC = time.time()
    
    # The simulation in executed here for EndTime/NumFrames times
    for jj in range(int(EndTime/dT/NumPlots)):
        program.SimulationKernel(queue, global_size, local_size, A_g, M_g)

    TimePointG = time.time()
    CompuTime += (TimePointG - TimePointC)

    # Get the data from the GPU
    cl.enqueue_copy(queue, A_host, A_g)
    cl.enqueue_copy(queue, M_host, M_g)
    
    # Updating the graphs
    im1.set_data(M_host.reshape(Size,Size))
    im2.set_data(A_host.reshape(Size,Size))
    plt.draw()
    plt.pause(1e-8)
    text.set_text("Time: %1.0f of %1.0f" % ((ii+1)/NumPlots*EndTime*Phi/24, EndTime*Phi/24))
    f1.savefig('fig/'+'Mussel'+"{:04d}".format(ii)+'.png', bbox_inches='tight')
    GraphTime += (time.time() - TimePointG)

# %% Wrapping up the simulation

# Determining the time that we used for the simulation
elapsed_time = time.time() - start_time    
print(" Simulation took      : %1.1f (s)" % (elapsed_time))
print("   Computation time   : %1.1f (s)" % CompuTime)
print("   Visualization time : %1.1f (s)\n" % GraphTime)

print(" Finished!\n")   

plt.close(f1)

ImportError: DLL load failed: 找不到指定的程序。